In [1]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 30.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

# 检查文件是否存在
file_path = '/content/drive/MyDrive/UTCI_prediction/data/spatial_images/landcover9.tif'
if os.path.exists(file_path):
    print("文件存在")
else:
    print("文件不存在，检查路径或文件名")

文件存在


读取并分割土地覆盖数据:

使用 read_tif 函数读取土地覆盖的 TIF 文件，返回一个包含土地利用数据的 NumPy 数组。 该数组被输入到 segment_by_landcover 函数中，这里使用 label 函数（来自 skimage.measure）对这些数据进行分割。label 函数会识别数组中的连续区域（或连通组件），其中每个唯一的区域（根据土地覆盖的不同值）被分配一个唯一的标签。


使用 regionprops 函数:

regionprops 接收来自 label 函数的分割后的标签数组（segmented_landcover），并生成一个包含每个独立区域（连通组件）的属性和统计信息的列表。 每个 region 对象包含有关其形状、大小、位置等的信息。region.label 是自动赋予该区域的唯一标识符（即 region_id）。 处理每个区域:

对于每个区域，使用 process_region 函数来提取相关的特征（如 UTCI 指数的平均值、最小值和最大值，以及 DEM 和土地覆盖的统计数据）。 region_id 用于唯一标识每个区域，因此在最终的数据集中可以追踪和区分不同的土地区域。

以下是第一轮的旧代码。
专注于从多个静态TIF文件（土地覆盖、树高、建筑高度和DEM）中读取数据，并结合一个CSV文件中的时间戳数据来生成特征。处理时间序列数据，主要是读取随时间变化的UTCI数据，并生成相关的时间序列特征。
这个程序会同时处理静态和动态数据，将时间戳与土地覆盖、树高、建筑高度和DEM数据结合起来，并处理时间序列的UTCI数据。以下是合并后的代码示例：

In [6]:
#第一轮的旧代码上半部分
import pandas as pd
import numpy as np
import rasterio
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu

def read_tif(file_path):
    """读取TIF文件并返回数据数组。"""
    with rasterio.open(file_path) as src:
        return src.read(1)  # 读取第一层数据

def segment_by_landcover(image):
    """Segment the landcover image based on its unique categorical values."""
    labels = label(image)  # Automatically label each area
    return labels

# 读取空间数据
landcover = read_tif('/content/drive/MyDrive/UTCI_prediction/data/spatial_images/landcover9.tif')
tree_height = read_tif('/content/drive/MyDrive/UTCI_prediction/data/spatial_images/tree height_9.tif')
building_height = read_tif('/content/drive/MyDrive/UTCI_prediction/data/spatial_images/building height_9.tif')
dem = read_tif('/content/drive/MyDrive/UTCI_prediction/data/spatial_images/DEM_9.tif')

segmented_landcover = segment_by_landcover(landcover)

# 读取CSV文件
csv_data = pd.read_csv('/content/drive/MyDrive/UTCI_prediction/data/Yifan_updated_data_with_timestamps.csv')

features_list = []
regions = regionprops(segmented_landcover, intensity_image=tree_height)

for index, row in csv_data.iterrows():
    for region in regions:
        features = {
            'region_id': region.label,
            'tree_height_mean': np.mean(tree_height[region.coords[:, 0], region.coords[:, 1]]),
            'building_height_mean': np.mean(building_height[region.coords[:, 0], region.coords[:, 1]]),
            'dem_mean': np.mean(dem[region.coords[:, 0], region.coords[:, 1]]),
            'timestamp': row['timestamp']
        }


        for col in csv_data.columns[1:]:
            features[col] = row[col]

        features_list.append(features)

all_features_df = pd.DataFrame(features_list)

# 将所有特征数据保存到一个CSV文件中
all_features_df.to_csv('/content/drive/MyDrive/UTCI_prediction/data/feature_output.csv', index=False)

#第一轮的旧代码
import os
import pandas as pd
import numpy as np
import rasterio
from skimage.measure import label, regionprops
from datetime import datetime, timedelta

def read_utci_from_tif(file_path):
    """专门用于从TIF文件读取UTCi数据的函数。"""
    with rasterio.open(file_path) as src:
        utci_data = src.read(1)
    return utci_data

def segment_by_landcover(image):
    """根据土地利用的唯一分类值进行分割。"""
    labels = label(image)
    return labels

# 读取土地利用数据并进行分割
landcover_path = '/content/drive/MyDrive/UTCI_prediction/data/spatial_images/landcover9.tif'
landcover = read_utci_from_tif(landcover_path)
segmented_landcover = segment_by_landcover(landcover)

# 新数据集文件夹路径
new_dataset_folder = '/content/drive/MyDrive/UTCI_prediction/data/output_images'
features_list = []


initial_timestamp = datetime.strptime("2022-07-03-00", "%Y-%m-%d-%H")
time_increment = timedelta(hours=1)

# 遍历新数据集中的每个文件
for index, filename in enumerate(sorted(os.listdir(new_dataset_folder))):
    if filename.endswith('.tif'):
        new_data_path = os.path.join(new_dataset_folder, filename)
        new_data = read_utci_from_tif(new_data_path)


        current_timestamp = initial_timestamp + index * time_increment
        timestamp_str = current_timestamp.strftime("%Y-%m-%d-%H")

        regions = regionprops(segmented_landcover, intensity_image=new_data)
        for region in regions:
            utc_values = new_data[region.coords[:, 0], region.coords[:, 1]]
            utc_mean = np.mean(utc_values) if utc_values.size > 0 else np.nan

            features = {
                'region_id': region.label,
                'UTCI': utc_mean,
                'timestamp': timestamp_str
            }

            features_list.append(features)

all_features_df = pd.DataFrame(features_list)
output_csv_path = '/content/drive/MyDrive/UTCI_prediction/data/old_method_UTCI_feature.csv'
all_features_df.to_csv(output_csv_path, index=False)

In [7]:
import pandas as pd

# 读取数据
input_data = pd.read_csv('/content/drive/MyDrive/UTCI_prediction/data/feature_output.csv')
utci_data = pd.read_csv('/content/drive/MyDrive/UTCI_prediction/data/old_method_UTCI_feature.csv')

# 基于 region_id 和 timestamp 列合并数据
merged_data = pd.merge(input_data, utci_data, on=['region_id', 'timestamp'], how='outer')

# 保存合并后的数据到 CSV 文件
merged_data.to_csv('/content/drive/MyDrive/UTCI_prediction/data/old_method_all_features.csv', index=False)

以下是新的数值计算部分

In [9]:
#0905修改的代码
import os
import pandas as pd
import numpy as np
import rasterio
from skimage.measure import label, regionprops
from datetime import datetime, timedelta

def read_tif(file_path):
    """
    读取TIF文件并返回数据数组。
    :param file_path: TIF文件的路径
    :return: 包含TIF数据的numpy数组
    """
    with rasterio.open(file_path) as src:
        return src.read(1)  # 读取第一个波段

def segment_by_landcover(image):
    """
    根据土地利用的唯一分类值进行分割。
    :param image: 土地利用数据数组
    :return: 分割后的标签数组
    """
    return label(image)

def process_region(region, utci_data, landcover_data, dem_data, timestamp):
    """
    处理单个区域并提取特征。
    :param region: 区域对象
    :param utci_data: UTCI数据数组
    :param landcover_data: 土地利用数据数组
    :param dem_data: DEM数据数组
    :param timestamp: 时间戳
    :return: 包含该区域特征的字典
    """
    coords = region.coords
    utci_values = utci_data[coords[:, 0], coords[:, 1]]
    landcover_values = landcover_data[coords[:, 0], coords[:, 1]]
    dem_values = dem_data[coords[:, 0], coords[:, 1]]

    # 转换土地利用数据为整数
    landcover_values_int = landcover_values.astype(int)

    return {
        'region_id': region.label,
        'UTCI_mean': np.mean(utci_values) if utci_values.size > 0 else np.nan,
        'UTCI_min': np.min(utci_values) if utci_values.size > 0 else np.nan,
        'UTCI_max': np.max(utci_values) if utci_values.size > 0 else np.nan,
        'main_landcover': np.argmax(np.bincount(landcover_values_int.flatten())),
        'dem_mean': np.mean(dem_values) if dem_values.size > 0 else np.nan,
        'dem_min': np.min(dem_values) if dem_values.size > 0 else np.nan,
        'dem_max': np.max(dem_values) if dem_values.size > 0 else np.nan,
        'timestamp': timestamp
    }


# 主程序
def main():
    # 设置文件路径
    landcover_path = '/content/drive/MyDrive/UTCI_prediction/data/spatial_images/landcover9.tif'
    dem_path = '/content/drive/MyDrive/UTCI_prediction/data/spatial_images/DEM_9.tif'
    utci_folder = '/content/drive/MyDrive/UTCI_prediction/data/output_images'
    output_csv_path = '/content/drive/MyDrive/UTCI_prediction/data/0905_feature.csv'

    # 读取土地利用和DEM数据
    print("读取土地利用和DEM数据...")
    landcover_data = read_tif(landcover_path)
    dem_data = read_tif(dem_path)

    # 对土地利用数据进行分割
    print("对土地利用数据进行分割...")
    segmented_landcover = segment_by_landcover(landcover_data)

    # 初始化特征列表和时间戳
    features_list = []
    initial_timestamp = datetime.strptime("2022-07-03-00", "%Y-%m-%d-%H")
    time_increment = timedelta(hours=1)

    # 处理UTCI数据文件
    print("处理UTCI数据文件...")
    for index, filename in enumerate(sorted(os.listdir(utci_folder))):
        if filename.endswith('.tif'):
            utci_path = os.path.join(utci_folder, filename)
            utci_data = read_tif(utci_path)

            current_timestamp = initial_timestamp + index * time_increment
            timestamp_str = current_timestamp.strftime("%Y-%m-%d-%H")

            regions = regionprops(segmented_landcover)
            for region in regions:
                features = process_region(region, utci_data, landcover_data, dem_data, timestamp_str)
                features_list.append(features)

        if (index + 1) % 10 == 0:
            print(f"已处理 {index + 1} 个文件...")

    # 创建DataFrame并保存为CSV
    print("创建DataFrame并保存为CSV...")
    all_features_df = pd.DataFrame(features_list)
    all_features_df.to_csv(output_csv_path, index=False)
    print(f"数据已保存至 {output_csv_path}")

if __name__ == "__main__":
    main()

读取土地利用和DEM数据...
对土地利用数据进行分割...
处理UTCI数据文件...
已处理 10 个文件...
已处理 20 个文件...
已处理 30 个文件...
已处理 40 个文件...
已处理 50 个文件...
已处理 60 个文件...
已处理 70 个文件...
已处理 80 个文件...
已处理 90 个文件...
已处理 100 个文件...
已处理 110 个文件...
已处理 120 个文件...
已处理 130 个文件...
已处理 140 个文件...
已处理 150 个文件...
已处理 160 个文件...
已处理 170 个文件...
已处理 180 个文件...
已处理 190 个文件...
已处理 200 个文件...
已处理 210 个文件...
已处理 220 个文件...
已处理 230 个文件...
已处理 240 个文件...
已处理 250 个文件...
已处理 260 个文件...
已处理 270 个文件...
已处理 280 个文件...
已处理 290 个文件...
已处理 300 个文件...
已处理 310 个文件...
已处理 320 个文件...
已处理 330 个文件...
创建DataFrame并保存为CSV...
数据已保存至 /content/drive/MyDrive/UTCI_prediction/data/0905_feature.csv


In [13]:
import pandas as pd

# 读取数据
input_data = pd.read_csv('/content/drive/MyDrive/UTCI_prediction/data/0905_feature.csv')
utci_data = pd.read_csv('/content/drive/MyDrive/UTCI_prediction/data/Yifan_updated_data_with_timestamps.csv')

# 基于 region_id 和 timestamp 列合并数据
merged_data = pd.merge(input_data, utci_data, on=['timestamp'], how='outer')

# 保存合并后的数据到 CSV 文件
merged_data.to_csv('/content/drive/MyDrive/UTCI_prediction/data/0905_merged_features.csv', index=False)